---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

In [5]:
import pandas as pd
import numpy as np

### Question 1
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable's]`

Convert the energy supply and the energy supply per capita to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102), which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country.

*This function should return a DataFrame with 20 columns and 15 entries.*

In [54]:
def energy():
    energy = pd.read_excel('Energy Indicators.xls',skiprows=17,na_values='...',skip_footer=38,parse_cols=[2,3,4,5,6] 
                       ,names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000
    dicts = {'Republic of Korea': 'South Korea',
               'United States of America': 'United States',
               'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
               'China, Hong Kong Special Administrative Region': 'Hong Kong'}
    energy['Country_M'] = energy['Country']
    energy = energy.set_index('Country')
    
    for country in energy.index:
        if (country.endswith(('1','2','3','4','5','6','7','8','9','0'))):
            country_edit = country[:-1]
            energy.set_value(country,'Country_M',country_edit)
            if (country_edit.endswith(('1','2','3','4','5','6','7','8','9','0'))):
                country_edit = country_edit[:-1]
                #print(country_edit)
                energy.set_value(country,'Country_M',country_edit)
    
    energy['Country_E'] = energy['Country_M']
    energy = energy.set_index('Country_M')
    #print(energy.columns)
    
    for country in energy.index:
        for key,value in dicts.items():
            #print('country>>>>',country)
            #print('key>>>>',key)
            if (key==country):
                energy.set_value(key,'Country_E',value)
        
    # removing () from Country names
    for names in energy.index:
        if names.find('(') > -1 or names.find(')') >-1:
            #print(names[:names.find('(')])
            energy.set_value(names,'Country_E',names[:names.find('(')].strip())
    energy.reset_index(drop=True,inplace=True)
    energy.columns = ['Energy Supply', 'Energy Supply per Capita', '% Renewable','Country']
    return energy

In [55]:
def gdp():
    GDP = pd.read_csv('world_bank.csv',skiprows=4)
    dicts = {'Korea, Rep.': 'South Korea', 
           'Iran, Islamic Rep.': 'Iran',
           'Hong Kong SAR, China': 'Hong Kong'}
    GDP['Country_M'] = GDP['Country Name']
    GDP = GDP.set_index('Country Name')
    for country in GDP.index:
        for key,value in dicts.items():
            if(country==key):
                GDP.set_value(country,'Country_M',value)
    GDP.reset_index(drop=True,inplace=True)
    GDP.columns = ['Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961',
           '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
           '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
           '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
           '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
           '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
           '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
           'Country']
    GDP_col_needed = list(np.arange(2006,2016,1))
    GDP_col_needed = list(map(str,GDP_col_needed))
    GDP_col_needed.append('Country')
    GDP_DF2 = GDP[GDP_col_needed]
    return GDP_DF2

In [56]:
def scimen():
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    return ScimEn

In [57]:
def answer_one():
    energy_df, GDP_df, ScimEn_df = energy(),gdp(),scimen()
    tdf = pd.merge(energy_df,GDP_df,how='inner',left_on='Country',right_on='Country')
    tdf = pd.merge(tdf,ScimEn_df,how='inner',left_on='Country',right_on='Country')
    condition_mask = tdf['Rank'] <= 15
    res = tdf[condition_mask]
    res = res.set_index('Country')
    return res

In [61]:
a = answer_one()
a.shape

(15, 20)

In [62]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, Australia to United States
Data columns (total 20 columns):
Energy Supply               15 non-null float64
Energy Supply per Capita    15 non-null float64
% Renewable                 15 non-null float64
2006                        15 non-null float64
2007                        15 non-null float64
2008                        15 non-null float64
2009                        15 non-null float64
2010                        15 non-null float64
2011                        15 non-null float64
2012                        15 non-null float64
2013                        15 non-null float64
2014                        15 non-null float64
2015                        14 non-null float64
Rank                        15 non-null int64
Documents                   15 non-null int64
Citable documents           15 non-null int64
Citations                   15 non-null int64
Self-citations              15 non-null int64
Citations per document      1

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [1]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [63]:
def answer_two():
    energy_df, GDP_df, ScimEn_df = energy(),gdp(),scimen()
    tdf_in = pd.merge(energy_df,GDP_df,how='inner',left_on='Country',right_on='Country')
    tdf_in = pd.merge(tdf_in,ScimEn_df,how='inner',left_on='Country',right_on='Country')
    tdf_out = pd.merge(energy_df,GDP_df,how='outer',left_on='Country',right_on='Country')
    tdf_out = pd.merge(tdf_out,ScimEn_df,how='outer',left_on='Country',right_on='Country')
    diff_count = len(tdf_out) - len(tdf_in)
    return diff_count

### Question 3 (6.6%)
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [64]:
def answer_three():
    Top15 = answer_one().copy()
    col_selected = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    Top15 = Top15[col_selected]
    result = Top15.apply(lambda x: np.mean(x),axis=1)
    result = result.sort_values(ascending=False)
    return result

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [99]:
def answer_four():
    Top15 = answer_one().copy()
    Top15 = Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    Top15['Avg_GDP'] = Top15.apply(np.mean,axis=1)
    Top15.sort_values(by='Avg_GDP',ascending=False,inplace=True)
    res_series = Top15.iloc[5]
    result = res_series.loc['2015'] - res_series.loc['2006']
    return result

### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [50]:
def answer_five():
    Top15 = answer_one()
    mean_perCapita = np.mean(Top15['Energy Supply per Capita'],axis=0)
    return mean_perCapita

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [66]:
def answer_six():
    Top15 = answer_one()
    return (Top15['% Renewable'].idxmax(),Top15['% Renewable'].max())

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [67]:
def answer_seven():
    Top15 = answer_one().copy()
    Top15 = Top15[['Citations','Self-citations']]
    Top15['Ration_Citation'] = np.divide(Top15['Self-citations'],Top15['Citations'])
    ans = (Top15['Ration_Citation'].idxmax(), Top15['Ration_Citation'].max())
    return ans

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [69]:
def answer_eight():
    Top15 = answer_one().copy()
    Top15['Population'] = np.divide(Top15['Energy Supply'],Top15['Energy Supply per Capita'])
    Pop_Sorted = Top15['Population'].sort_values(ascending=False)
    return Pop_Sorted.index[2]

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita?

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita).*

In [100]:
def answer_nine():
    Top15 = answer_one().copy()
    Top15['Population'] = np.divide(Top15['Energy Supply'],Top15['Energy Supply per Capita'])
    Top15['Citable_Doc_Per_Capita'] = np.divide(Top15['Citable documents'],Top15['Population'])
    Top15 = Top15[['Energy Supply per Capita','Citable_Doc_Per_Capita']]
    cor_value = Top15.corr()
    result = cor_value.iloc[1][0]
    return result

In [11]:
def plot9():    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [12]:
#plot9()

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [95]:
def answer_ten():
    Top15 = answer_one()
    renew_mean = Top15['% Renewable'].mean()
    def funct1(x):
        if x['% Renewable'] >= renew_mean:
            return 1
        else:
            return 0
    Top15['New_Col'] = Top15.apply(funct1,axis=1)
    Top15 = Top15.sort_values(by='Rank')
    return Top15['New_Col']

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [97]:
def answer_eleven():
    Top15 = answer_one().copy()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df_q11 = pd.Series(ContinentDict)
    df = pd.concat((Top15,df_q11),join='inner',axis=1)
    df=df.reset_index('Continent')
    df.columns = [                  'Country',            'Energy Supply',
                   'Energy Supply per Capita',              '% Renewable',
                                       '2006',                     '2007',
                                       '2008',                     '2009',
                                       '2010',                     '2011',
                                       '2012',                     '2013',
                                       '2014',                     '2015',
                                       'Rank',                'Documents',
                          'Citable documents',                'Citations',
                             'Self-citations',   'Citations per document',
                                    'H index',                'Continent']
    df = df.set_index('Continent')
    df['Population'] = np.divide(df['Energy Supply'],df['Energy Supply per Capita'])
    df_final = df[['Country','Population']].copy()
    res = df_final.groupby(level=0)['Population'].agg({'size':np.count_nonzero,'sum': np.sum,'mean': np.average,'std': np.std})
    return res

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [15]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas)

e.g. 12345678.90 -> 12,345,678.90

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [16]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [17]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [18]:
#plot_optional()